In [1]:
import pandas as pd
import os
from dataclasses import dataclass, field
import sys
sys.path.insert(0, '/home/jovyan/work')
sys.path.insert(1, '/home/jovyan/work/src')
sys.path.insert(2, '/home/jovyan/work/src/service')

from service import bfs_cases_db_service as bfs_db

from py.global_configs import *
from py.normalize import normalize

/home/jovyan/work/src/models/BfsCase.py:5: SAWarning: On class 'BfsCase', Column object 'drg_cost_weight' named directly multiple times, only one will be used: drg_cost_weight, effective_cost_weight. Consider using orm.synonym instead
  class BfsCase(Base):


In [2]:
# check all the file name

FILES_TO_ANALYZE.keys()


dict_keys(['Hirslanden Salem 2017', 'Hirslanden Beau Site 2017', 'Hirslanden Linde 2017', 'Hirslanden Linde 2018', 'Hirslanden Salem 2018', 'Hirslanden Beau Site 2018', 'Winterthur 2017', 'Winterthur 2018', 'Winterthur 2019', 'Winterthur 2020', 'USZ 2018', 'USZ 2019', 'USZ 2019_30_04_2020', 'KSSG 2021', 'KSSG Linth 2019', 'KSSG Toggenburg 2019', 'KSSG SRRWS 2019'])

In [3]:
file = FILES_TO_ANALYZE['KSSG Toggenburg 2019']
file

FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/Linth_Toggenburg_SRRWS_2019.xlsx', hospital_name_db='KSSG Toggenburg', year='2019', sheets=['Änderungen_Toggenburg_2019'])

In [4]:

df_revised_case_d2d = normalize(file, 0)

Read 28 cases for KSSG Toggenburg 2019
TYPES:
case_id             string
patient_id          string
gender              string
age_years            int64
duration_of_stay     int64
pccl                 int64
drg                 string
pd                  string
bfs_code            string
added_icds          string
removed_icds        string
added_chops         string
removed_chops       string
dtype: object


In [6]:
df_revised_case_d2d.head()

,case_id,patient_id,gender,age_years,duration_of_stay,pccl,drg,pd,bfs_code,added_icds,removed_icds,added_chops,removed_chops
0,4408298963,3FEA1A97AEC65660,W,78,6,2,I68E,S2206,M100,I652,<NA>,<NA>,<NA>
1,4450072153,81D216BDF5F6A401,M,78,9,3,G07Z,C20,M200,K921,<NA>,<NA>,<NA>
2,4408295683,63DA4A8CA1E4B8E4,M,69,15,3,F13C,E1175,M200,M8607,M8687,<NA>,<NA>
3,4408299930,F5DECA545F5CCA09,W,83,13,3,G17Z,K5732,M200,"K5722,E870",K5732,<NA>,<NA>
4,4408313061,0B77F24272929EBB,M,59,9,3,H09C,K8510,M200,K8040,K819,<NA>,<NA>


# match to the database


In [7]:
# get the case_id from revised_case

revised_case_id = df_revised_case_d2d['case_id'].values
revised_case_id

array(['4408298963', '4450072153', '4408295683', '4408299930',
       '4408313061', '4408290225', '4408299655', '4408294065',
       '4408296873', '4408300426', '4408309452', '4408313863',
       '4450069712', '4408288172', '4408288632', '4408309949',
       '4408308749', '4408291185', '4408300943', '4408305935',
       '4450066769', '4408315954', '4408291872', '4408296278',
       '4408301545', '4408308593', '4408295558', '4408312516'],
      dtype=object)

In [8]:
# match to the database
revised_case_db = bfs_db.get_bfs_cases_by_ids(revised_case_id)
revised_case_db.head()

,drg_cost_weight,aimedic_id,hospital_id,case_id,patient_id,age_years,age_days,gender,duration_of_stay,clinic_id,ventilation_hours,admission_weight,gestation_age,admission_date,admission_type,discharge_date,discharge_type,drg,adrg,pccl
0,0.502,247092,9,4408288172,C783459AA87181DB,84,0,M,6,3,0,0,0,2019-01-09,01,2019-01-15,00,F72B,F72,2
1,0.448,246129,9,4408288632,FE6285F488AB1D73,78,0,M,6,3,0,0,0,2019-01-13,01,2019-01-19,00,G72C,G72,3
2,0.888,242149,9,4408290225,0FCD4AF3123CA5C9,61,0,M,5,3,0,0,0,2019-02-01,01,2019-02-06,00,F62C,F62,3
3,0.629,248082,9,4408291185,7273BEEA751CDBD5,88,0,W,8,3,0,0,0,2019-02-12,01,2019-02-20,00,E77F,E77,2
4,0.482,243033,9,4408291872,E3855FA45E63A196,59,0,M,2,3,0,0,0,2019-02-19,01,2019-02-21,00,E69C,E69,3


In [9]:
# 
print('There are {} out of {} revised cases from DtoD that are matched with the database for {} {}'.format(len(revised_case_db), len(revised_case_id), file.hospital_name_db, file.year))

There are 28 out of 28 revised cases from DtoD that are matched with the database for KSSG Toggenburg 2019


In [10]:
# if we find the match cases, then we need to check if the case_id, gender, year....are match